<a href="https://colab.research.google.com/github/ElBenjaM/VHS/blob/main/Extractor_de_Frames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 🔽 Paso 1: Conectar Google Drive
#@markdown ---
#@markdown ### 📂 Montar Google Drive
#@markdown Ejecuta esta celda para conectar tu Google Drive y poder acceder a tus archivos.

from google.colab import drive

# --- Montar Google Drive ---
try:
    drive.mount('/content/drive')
    print("\n✅ Google Drive conectado correctamente.")
    print("➡️ Ahora puedes proceder a ejecutar la celda del 'Paso 2'.")
except Exception as e:
    print(f"❌ Error al conectar con Google Drive: {e}")


In [ ]:
#@title 🎞️ Paso 2: Extraer Frames (Acelerado por GPU y Guardado en Drive)
#@markdown ---
#@markdown ### 🎬 Define los parámetros y ejecuta
#@markdown 1.  Escribe el **nombre exacto** de tu video (incluyendo la extensión, ej: `mi_video.mp4`).
#@markdown 2.  El código lo buscará y guardará los frames en una **nueva carpeta junto a tu video en Google Drive**.
#@markdown 3.  Ajusta los FPS y haz clic en 'Extraer Frames'.

import os
import ipywidgets as widgets
from IPython.display import display, clear_output
import shutil
import subprocess

# --- Interfaz de Usuario ---
style = {'description_width': 'initial'}

video_name_widget = widgets.Text(
    value='tu_video.mp4',
    placeholder='Escribe el nombre del video aquí',
    description='Nombre del video:',
    style=style,
    layout=widgets.Layout(width='80%')
)

fps_widget = widgets.IntSlider(
    value=30,
    min=1,
    max=60,
    step=1,
    description='Frames por Segundo (FPS):',
    style=style
)

run_button = widgets.Button(
    description='🎬 Extraer Frames',
    button_style='success',
    tooltip='Buscar el video y empezar la extracción',
    icon='play'
)

output_area = widgets.Output()

def is_gpu_available():
    """Verifica si Colab tiene una GPU NVIDIA disponible."""
    try:
        result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
        return result.returncode == 0
    except FileNotFoundError:
        return False

def find_video_in_drive(file_name):
    """Busca un archivo en /content/drive/MyDrive y devuelve su ruta completa."""
    print(f"🔎 Buscando '{file_name}' en tu Google Drive. Esto puede tardar un momento...")
    for root, dirs, files in os.walk('/content/drive/MyDrive'):
        if file_name in files:
            found_path = os.path.join(root, file_name)
            print(f"✅ ¡Video encontrado en: {found_path}!")
            return found_path
    return None

def display_ui():
    """Muestra la interfaz de usuario."""
    clear_output(wait=True)
    print("📂 Introduce el nombre de tu video y selecciona los FPS deseados.")
    if not is_gpu_available():
        print("\n⚠️ ADVERTENCIA: No se detectó una GPU. El proceso se ejecutará en el CPU y podría ser lento.")
        print("   Para acelerarlo, ve a 'Entorno de ejecución' > 'Cambiar tipo de entorno de ejecución' y selecciona 'T4 GPU'.")
    else:
        print("\n✅ GPU T4 detectada. El proceso será acelerado.")
    display(video_name_widget, fps_widget, run_button, output_area)
    run_button.on_click(on_run_button_clicked)

def on_run_button_clicked(b):
    """Función que se ejecuta al presionar el botón."""
    with output_area:
        clear_output(wait=True)
        video_name = video_name_widget.value
        target_fps = fps_widget.value

        video_path = find_video_in_drive(video_name)

        if not video_path:
            print(f"❌ ERROR: No se pudo encontrar el video con el nombre '{video_name}' en tu Google Drive.")
            print("Verifica que el nombre y la extensión (ej: .mp4) sean correctos.")
            return

        extract_frames_ffmpeg(video_path, target_fps)

def extract_frames_ffmpeg(video_path, target_fps):
    """
    Función principal para desarmar el video en frames usando ffmpeg.
    Guarda los frames en una nueva carpeta dentro de Google Drive.
    """
    # --- MODIFICACIÓN PARA PRUEBAS ---
    # Cambia esta variable a 'False' para procesar el video completo.
    # Si está en 'True', solo procesará los primeros 15 segundos.
    MODO_PRUEBA_15_SEGUNDOS = True

    print(f"\n▶️ Iniciando proceso para el video: {os.path.basename(video_path)}")
    print(f"🎯 Objetivo: Extraer {target_fps} frames por segundo.")

    video_directory = os.path.dirname(video_path)
    video_filename = os.path.basename(video_path)
    video_name, _ = os.path.splitext(video_filename)
    output_folder_path = os.path.join(video_directory, f"{video_name}_frames")

    print(f"💾 La carpeta de salida se creará en Google Drive: '{output_folder_path}'")

    if os.path.exists(output_folder_path):
        print(f"🧹 Limpiando carpeta existente en Google Drive...")
        shutil.rmtree(output_folder_path)
    os.makedirs(output_folder_path)
    print(f"📁 Carpeta de salida creada en Google Drive.")

    try:
        output_pattern = os.path.join(output_folder_path, "frame_%06d.jpg")

        # Construir el comando de ffmpeg
        if is_gpu_available():
            print("🚀 Usando aceleración por GPU (NVIDIA HWACCEL)...")
            command = ['ffmpeg', '-hwaccel', 'cuda', '-i', video_path]
        else:
            print("🐌 Usando CPU... (Puede ser lento para videos grandes)")
            command = ['ffmpeg', '-i', video_path]

        # --- AÑADIR LÍMITE DE TIEMPO SI ESTÁ EN MODO PRUEBA ---
        if MODO_PRUEBA_15_SEGUNDOS:
            print("\n⚠️ MODO DE PRUEBA ACTIVO: Solo se procesarán los primeros 15 segundos del video.")
            print("   Para procesar el video completo, edita esta celda y cambia 'MODO_PRUEBA_15_SEGUNDOS' a 'False'.\n")
            command.extend(['-t', '45'])

        # Añadir el resto de los parámetros
        command.extend(['-vf', f'fps={target_fps}', '-q:v', '2', output_pattern])

        print("\n⏳ Extrayendo frames con FFMPEG. Puedes ver el progreso abajo:")
        subprocess.run(command, check=True, stderr=subprocess.PIPE, text=True)

        saved_frame_count = len(os.listdir(output_folder_path))

        print(f"\n🎉 ¡Proceso completado!")
        print(f"Se han guardado {saved_frame_count} frames en la carpeta '{os.path.basename(output_folder_path)}'.")
        print("✨ Puedes encontrar la nueva carpeta en tu Google Drive, junto al video original.")

    except subprocess.CalledProcessError as e:
        print(f"❌ Ocurrió un error durante el procesamiento con FFMPEG.")
        print("--- Mensaje de error de FFMPEG ---")
        print(e.stderr)
    except Exception as e:
        print(f"❌ Ocurrió un error inesperado: {e}")


# --- Mostrar la interfaz al ejecutar la celda ---
if 'google.colab' in str(get_ipython()):
    display_ui()
else:
    print("Este script está diseñado para ejecutarse en Google Colab.")



In [ ]:
#@title 🎯 Paso 3 (Definitivo): Detector Configurable
#@markdown ---
#@markdown ### 🏆 Detector final con dos modos de operación.
#@markdown **Modo Manual:** Ajusta los umbrales tú mismo con los sliders.
#@markdown **Configurar con Prompt:** Pega el prompt generado por el Laboratorio para una configuración automática y precisa.
#@markdown ---

import os
import cv2
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
from glob import glob
import shutil
import re

# --- Interfaz de Usuario ---
style = {'description_width': 'initial'}

folder_name_widget_final = widgets.Text(value='tu_video_frames', placeholder='Escribe el nombre de la carpeta', description='Carpeta de Frames en Drive:', style=style, layout=widgets.Layout(width='90%'))

mode_selection = widgets.RadioButtons(options=['Modo Manual', 'Configurar con Prompt'], description='Modo de Configuración:')

# --- Controles Manuales ---
avg_sat_threshold_widget = widgets.FloatSlider(value=94.0, min=70.0, max=120.0, step=0.5, description='1. Umbral Saturación Media (<):', style=style, readout_format='.1f')
low_sat_threshold_widget = widgets.FloatSlider(value=30.0, min=15.0, max=50.0, step=0.5, description='2. Umbral Baja Saturación (>):', style=style, readout_format='.1f')
manual_controls = widgets.VBox([avg_sat_threshold_widget, low_sat_threshold_widget])

# --- Controles por Prompt ---
prompt_text_widget = widgets.Text(value="avg_sat < 94.0; low_sat > 30.0", placeholder='Pega aquí el prompt del laboratorio', description='Prompt:', style=style, layout=widgets.Layout(width='90%'))
prompt_controls = widgets.VBox([prompt_text_widget])

run_button_final = widgets.Button(description='🎯 Ejecutar Detector', button_style='success', tooltip='Empezar el análisis de doble métrica', icon='check')
output_area_final = widgets.Output()

def display_final_ui():
    clear_output(wait=True)
    display(folder_name_widget_final, mode_selection, manual_controls, run_button_final, output_area_final)
    mode_selection.observe(on_mode_change, names='value')

def on_mode_change(change):
    clear_output(wait=True)
    display(folder_name_widget_final, mode_selection)
    if change.new == 'Modo Manual':
        display(manual_controls)
    else:
        display(prompt_controls)
    display(run_button_final, output_area_final)

def on_run_button_final_clicked(b):
    with output_area_final:
        clear_output(wait=True)
        folder_name = folder_name_widget_final.value

        # --- Lógica de Configuración ---
        if mode_selection.value == 'Modo Manual':
            avg_sat_thresh = avg_sat_threshold_widget.value
            low_sat_thresh = low_sat_threshold_widget.value
            print("Iniciando en Modo Manual...")
        else: # Modo Prompt
            print("Iniciando con configuración de Prompt...")
            try:
                prompt = prompt_text_widget.value
                parts = prompt.replace(" ", "").split(';')
                avg_sat_part = [p for p in parts if 'avg_sat' in p][0]
                low_sat_part = [p for p in parts if 'low_sat' in p][0]

                avg_sat_thresh = float(re.search(r'(\d+\.?\d*)', avg_sat_part).group(1))
                low_sat_thresh = float(re.search(r'(\d+\.?\d*)', low_sat_part).group(1))
                print(f"  - Umbral de Saturación Media (<) establecido en: {avg_sat_thresh}")
                print(f"  - Umbral de Baja Saturación (>) establecido en: {low_sat_thresh}")
            except Exception as e:
                print(f"❌ ERROR: El formato del prompt es incorrecto. Asegúrate de que sea como 'avg_sat < 94.0; low_sat > 30.0'. Error: {e}")
                return

        frames_path = find_folder_in_drive_final(folder_name)
        if not frames_path: print(f"❌ ERROR: No se encontró la carpeta '{folder_name}'."); return
        analyze_frames_final(frames_path, avg_sat_thresh, low_sat_thresh)

def find_folder_in_drive_final(folder_name):
    print(f"🔎 Buscando la carpeta '{folder_name}'...")
    for root, dirs, files in os.walk('/content/drive/MyDrive'):
        if folder_name in dirs:
            found_path = os.path.join(root, folder_name); print(f"✅ ¡Carpeta encontrada!"); return found_path
    return None

def analyze_frames_final(frames_path, avg_sat_threshold, low_sat_threshold):
    print(f"\n▶️ Analizando frames con la configuración aplicada...")
    image_files = sorted(glob(os.path.join(frames_path, '*.*')))
    if not image_files: print("❌ No se encontraron imágenes."); return

    corrupt_folder_path = os.path.join(frames_path, '_corruptos_definitivo')
    os.makedirs(corrupt_folder_path, exist_ok=True)
    corrupt_count = 0
    progress_bar = widgets.IntProgress(value=0, min=0, max=len(image_files), description='Progreso:')
    display(progress_bar)

    for i, img_path in enumerate(image_files):
        try:
            img = cv2.imread(img_path)
            if img is None: continue
            hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
            avg_saturation = np.mean(hsv[:, :, 1])
            low_sat_mask = cv2.inRange(hsv, np.array([0, 0, 0]), np.array([180, 50, 255]))
            low_sat_percentage = (cv2.countNonZero(low_sat_mask) * 100) / (img.shape[0] * img.shape[1])

            if avg_saturation < avg_sat_threshold and low_sat_percentage > low_sat_threshold:
                corrupt_count += 1
                shutil.move(img_path, os.path.join(corrupt_folder_path, os.path.basename(img_path)))
            progress_bar.value = i + 1
        except Exception as e:
            print(f"  - ⚠️ Error procesando {os.path.basename(img_path)}: {e}")

    progress_bar.description = 'Completado'
    print("\n🎉 ¡Análisis definitivo completado!")
    print(f"Total de frames corruptos detectados: {corrupt_count}")

if 'google.colab' in str(get_ipython()):
    run_button_final.on_click(on_run_button_final_clicked)
    display_final_ui()
else:
    print("Este script está diseñado para ejecutarse en Google Colab.")


In [ ]:
#@title ⏪ Paso 4 (Actualizado): Restaurar Todos los Frames
#@markdown ---
#@markdown ### 🔄 Mueve los frames de CUALQUIER carpeta de corruptos de vuelta a la principal.
#@markdown Busca `_corruptos` y `_corruptos_por_bandas` y restaura todos los archivos encontrados.
#@markdown 1.  Escribe el nombre de la carpeta de frames principal (ej: `mi_video_frames`).
#@markdown 2.  Ejecuta para poder volver a analizar con diferentes ajustes.

import os
import ipywidgets as widgets
from IPython.display import display, clear_output
from glob import glob
import shutil

# --- Interfaz de Usuario ---
style = {'description_width': 'initial'}

folder_name_widget_restore = widgets.Text(
    value='tu_video_frames',
    placeholder='Escribe el nombre de la carpeta de frames',
    description='Carpeta de Frames en Drive:',
    style=style,
    layout=widgets.Layout(width='80%')
)

run_button_restore = widgets.Button(
    description='⏪ Restaurar Todos los Frames',
    button_style='info',
    tooltip='Mover todos los frames de las carpetas de corruptos a la principal',
    icon='undo'
)

output_area_restore = widgets.Output()

def find_folder_in_drive_restore(folder_name):
    """Busca una carpeta en /content/drive/MyDrive y devuelve su ruta completa."""
    print(f"🔎 Buscando la carpeta '{folder_name}' en tu Google Drive...")
    for root, dirs, files in os.walk('/content/drive/MyDrive'):
        if folder_name in dirs:
            found_path = os.path.join(root, folder_name)
            print(f"✅ ¡Carpeta encontrada en: {found_path}!")
            return found_path
    return None

def display_restore_ui():
    """Muestra la interfaz de usuario del restaurador."""
    clear_output(wait=True)
    print("📂 Introduce el nombre de la carpeta principal de frames para restaurar los archivos.")
    display(folder_name_widget_restore, run_button_restore, output_area_restore)
    run_button_restore.on_click(on_run_button_restore_clicked)

def on_run_button_restore_clicked(b):
    """Función que se ejecuta al presionar el botón de restaurar."""
    with output_area_restore:
        clear_output(wait=True)
        folder_name = folder_name_widget_restore.value

        frames_path = find_folder_in_drive_restore(folder_name)

        if not frames_path:
            print(f"❌ ERROR: No se pudo encontrar la carpeta principal con el nombre '{folder_name}'.")
            return

        restore_all_files(frames_path)

def restore_all_files(frames_path):
    """
    Busca las carpetas '_corruptos' y '_corruptos_por_bandas' y mueve sus contenidos a la carpeta padre.
    """
    print(f"\n▶️ Iniciando proceso de restauración para: {os.path.basename(frames_path)}")

    corrupt_folders_to_check = [
        os.path.join(frames_path, '_corruptos'),
        os.path.join(frames_path, '_corruptos_por_bandas')
    ]

    files_to_restore = []

    for folder_path in corrupt_folders_to_check:
        if os.path.exists(folder_path):
            print(f"  - Detectada carpeta: '{os.path.basename(folder_path)}'")
            files_in_folder = glob(os.path.join(folder_path, '*.*'))
            files_to_restore.extend(files_in_folder)

    if not files_to_restore:
        print("\n✅ No se encontraron archivos en ninguna de las carpetas de corruptos. No hay nada que restaurar.")
        return

    total_files = len(files_to_restore)
    print(f"\n⏳ Se moverán {total_files} frames de vuelta a la carpeta principal...")

    progress_bar = widgets.IntProgress(value=0, min=0, max=total_files, description='Restaurando:')
    display(progress_bar)

    moved_count = 0
    for file_path in files_to_restore:
        try:
            # Mover el archivo al directorio padre (frames_path)
            shutil.move(file_path, frames_path)
            moved_count += 1
            progress_bar.value += 1
        except Exception as e:
            print(f"  - ⚠️ Error al mover {os.path.basename(file_path)}: {e}")

    progress_bar.description = 'Completado'
    print(f"\n🎉 ¡Restauración completada!")
    print(f"Se han movido {moved_count} frames de vuelta a '{os.path.basename(frames_path)}'.")
    print("✨ Ahora puedes ejecutar el Paso 3 de nuevo con una sensibilidad diferente.")

# --- Mostrar la interfaz al ejecutar la celda ---
if 'google.colab' in str(get_ipython()):
    display_restore_ui()
else:
    print("Este script está diseñado para ejecutarse en Google Colab.")


In [ ]:
#@title 🔬 Laboratorio y Generador de Prompts
#@markdown ---
#@markdown ### ախ Sube hasta 10 frames buenos y 10 malos (o usa la lista predeterminada).
#@markdown Este laboratorio realizará un análisis completo y generará un **"Prompt de Configuración"** optimizado para tu video.
#@markdown **Copia el prompt generado y pégalo en el "Paso 3" para una detección perfecta.**
#@markdown ---

import os
import cv2
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import io

# --- Interfaz de Usuario ---
style = {'description_width': 'initial'}

# --- Sección para Carga Manual ---
good_frames_uploader = widgets.FileUpload(accept='image/*', description='1. Sube Frames BUENOS (hasta 10):', style=style, multiple=True)
good_frames_feedback = widgets.Output()
bad_frames_uploader = widgets.FileUpload(accept='image/*', description='2. Sube Frames MALOS (hasta 10):', style=style, multiple=True)
bad_frames_feedback = widgets.Output()
run_button_analyzer = widgets.Button(description='🔬 Analizar Frames Subidos', button_style='primary', tooltip='Analizar los frames que acabas de subir', icon='upload')

# --- Sección para Carga Predeterminada desde Drive ---
folder_name_widget_analyzer = widgets.Text(value='tu_video_frames', placeholder='Escribe el nombre de la carpeta', description='Carpeta de Frames en Drive:', style=style, layout=widgets.Layout(width='80%'))
run_button_predefined = widgets.Button(description='🧠 Usar Frames Predeterminados', button_style='danger', tooltip='Analizar la lista de frames predeterminada', icon='flask')

output_area_analyzer = widgets.Output()
prompt_output_area = widgets.Output()

def display_analyzer_ui():
    clear_output(wait=True)
    print("Sube tus ejemplos o usa la lista predeterminada para generar un prompt de configuración para el Paso 3.")
    print("\n--- Opción 1: Carga Manual ---")
    display(good_frames_uploader, good_frames_feedback, bad_frames_uploader, bad_frames_feedback, run_button_analyzer)
    print("\n--- Opción 2: Carga Predeterminada desde Drive ---")
    display(folder_name_widget_analyzer, run_button_predefined)
    display(output_area_analyzer, prompt_output_area)

    good_frames_uploader.observe(lambda change: on_file_upload(change, good_frames_feedback, "BUENOS"), names='value')
    bad_frames_uploader.observe(lambda change: on_file_upload(change, bad_frames_feedback, "MALOS"), names='value')
    run_button_analyzer.on_click(on_run_button_analyzer_clicked)
    run_button_predefined.on_click(on_run_button_predefined_clicked)

def on_file_upload(change, feedback_area, type_str):
    with feedback_area:
        clear_output(wait=True)
        if change['new']:
            filenames = sorted(list(change['new'].keys()))
            print(f"✅ {len(filenames)} frame(s) {type_str} cargado(s):")
            for name in filenames: print(f"  - {name}")

def on_run_button_analyzer_clicked(b):
    with output_area_analyzer:
        clear_output(wait=True)
        prompt_output_area.clear_output()
        if not good_frames_uploader.value or not bad_frames_uploader.value:
            print("❌ Por favor, sube al menos un frame BUENO y uno MALO."); return
        good_images = [cv2.imdecode(np.frombuffer(f['content'], np.uint8), cv2.IMREAD_COLOR) for f in good_frames_uploader.value.values()]
        bad_images = [cv2.imdecode(np.frombuffer(f['content'], np.uint8), cv2.IMREAD_COLOR) for f in bad_frames_uploader.value.values()]
        run_all_analyses(good_images, bad_images)

def find_folder_in_drive_analyzer(folder_name):
    print(f"🔎 Buscando la carpeta '{folder_name}'...")
    for root, dirs, files in os.walk('/content/drive/MyDrive'):
        if folder_name in dirs:
            found_path = os.path.join(root, folder_name)
            print(f"✅ ¡Carpeta encontrada en: {found_path}!"); return found_path
    return None

def on_run_button_predefined_clicked(b):
    with output_area_analyzer:
        clear_output(wait=True)
        prompt_output_area.clear_output()
        folder_name = folder_name_widget_analyzer.value
        frames_path = find_folder_in_drive_analyzer(folder_name)
        if not frames_path: print(f"❌ ERROR: No se encontró la carpeta '{folder_name}'."); return

        predefined_good = ['frame_000004.jpg', 'frame_000023.jpg', 'frame_000026.jpg', 'frame_000039.jpg', 'frame_000042.jpg', 'frame_000048.jpg', 'frame_000061.jpg', 'frame_000065.jpg', 'frame_000070.jpg', 'frame_000074.jpg']
        predefined_bad = ['frame_000005.jpg', 'frame_000024.jpg', 'frame_000027.jpg', 'frame_000040.jpg', 'frame_000043.jpg', 'frame_000049.jpg', 'frame_000062.jpg', 'frame_000066.jpg', 'frame_000071.jpg', 'frame_000075.jpg']

        print("⏳ Cargando frames predeterminados...")
        good_images = [cv2.imread(os.path.join(frames_path, f)) for f in predefined_good]
        bad_images = [cv2.imread(os.path.join(frames_path, f)) for f in predefined_bad]
        good_images = [img for img in good_images if img is not None]
        bad_images = [img for img in bad_images if img is not None]
        if not good_images or not bad_images: print("❌ ERROR: No se pudieron cargar algunos frames predeterminados."); return
        run_all_analyses(good_images, bad_images)

def get_saturation_mean(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    return np.mean(hsv[:, :, 1])

def get_low_saturation_percentage(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    low_sat_mask = cv2.inRange(hsv, np.array([0, 0, 0]), np.array([180, 50, 255]))
    return (cv2.countNonZero(low_sat_mask) * 100) / (img.shape[0] * img.shape[1])

def run_all_analyses(good_images, bad_images):
    print(f"--- 🧠 Informe de Análisis Profundo 🧠 ---\nAnalizando {len(good_images)} frames BUENOS y {len(bad_images)} frames MALOS.\n")

    good_sat_mean = [get_saturation_mean(img) for img in good_images]
    bad_sat_mean = [get_saturation_mean(img) for img in bad_images]

    good_low_sat_perc = [get_low_saturation_percentage(img) for img in good_images]
    bad_low_sat_perc = [get_low_saturation_percentage(img) for img in bad_images]

    # --- Generación de Prompt ---
    avg_sat_good_avg = np.mean(good_sat_mean)
    avg_sat_bad_avg = np.mean(bad_sat_mean)
    low_sat_good_avg = np.mean(good_low_sat_perc)
    low_sat_bad_avg = np.mean(bad_low_sat_perc)

    # El umbral es el punto medio entre los promedios de buenos y malos
    avg_sat_threshold = (avg_sat_good_avg + avg_sat_bad_avg) / 2
    low_sat_threshold = (low_sat_good_avg + low_sat_bad_avg) / 2

    prompt = f"avg_sat < {avg_sat_threshold:.1f}; low_sat > {low_sat_threshold:.1f}"

    with prompt_output_area:
        print("--- 📋 Prompt de Configuración Generado ---")
        print("Copia esta línea y pégala en el 'Paso 3' en el modo 'Configurar con Prompt'.")
        prompt_widget = widgets.Text(value=prompt, layout=widgets.Layout(width='90%'))
        display(prompt_widget)

if 'google.colab' in str(get_ipython()):
    display_analyzer_ui()
else:
    print("Este script está diseñado para ejecutarse en Google Colab.")
